In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import itertools
from tqdm import tqdm

import plotly.graph_objects as go

import numpy as np
import time

In [3]:
df = pd.read_excel("../data/Sorteios_Completo.xlsx")
df['sorteados_seq'] = df[['Bola 01','Bola 02','Bola 03','Bola 04','Bola 05','Bola 06','Bola 07','Bola 08','Bola 09','Bola 10','Bola 11','Bola 12','Bola 13','Bola 14','Bola 15']].values.tolist()

df.drop(['Bola 01','Bola 02','Bola 03','Bola 04','Bola 05','Bola 06','Bola 07','Bola 08','Bola 09','Bola 10','Bola 11','Bola 12','Bola 13','Bola 14','Bola 15','Id','Sorteado 01',
 'Sorteado 02',
 'Sorteado 03',
 'Sorteado 04',
 'Sorteado 05',
 'Sorteado 06',
 'Sorteado 07',
 'Sorteado 08',
 'Sorteado 09',
 'Sorteado 10',
 'Sorteado 11',
 'Sorteado 12',
 'Sorteado 13',
 'Sorteado 14',
 'Sorteado 15',
 'Arrecadação_Total',
 'Ganhadores 15 Números',
 'Ganhadores 14 Números',
 'Ganhadores 13 Números',
 'Ganhadores 12 Números',
 'Ganhadores 11 Números',
#  'Valor Rateio 15 Números',
#  'Valor Rateio 14 Números',
#  'Valor Rateio 13 Números',
#  'Valor Rateio 12 Números',
#  'Valor Rateio 11 Números',
 'Acumulado 15 Números',
 'Estimativa Prêmio',
 'Valor Acumulado Especial',
 'Local 15 Números'], axis=1, inplace=True)


df.head()

,Concurso,Data Sorteio,Valor Rateio 15 Números,Valor Rateio 14 Números,Valor Rateio 13 Números,Valor Rateio 12 Números,Valor Rateio 11 Números,sorteados_seq
0,1,2003-09-29,49765.82,689.84,10.0,4,2.0,"[2, 3, 5, 6, 9, 10, 11, 13, 14, 16, 18, 20, 23..."
1,2,2003-10-06,596323.70,1388.95,10.0,4,2.0,"[1, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 19, 20,..."
2,3,2003-10-13,400623.70,2173.36,10.0,4,2.0,"[1, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 20,..."
3,4,2003-10-20,902226.02,1498.72,10.0,4,2.0,"[1, 2, 4, 5, 8, 10, 12, 13, 16, 17, 18, 19, 23..."
4,5,2003-10-27,380017.55,687.49,10.0,4,2.0,"[1, 2, 4, 8, 9, 11, 12, 13, 15, 16, 19, 20, 23..."


In [4]:
df_seq = df.copy()
for i in range(1,26):
    if i < 10:
        num = '0'+str(i)
    else:
        num = str(i)
    colNamePos = 'seqPos'+num
    colNameNeg = 'seqNeg'+num
    df_seq[[colNamePos, colNameNeg]] = 0
# df.head()

In [5]:
for idx, row in df_seq.iterrows():
    for key in row.keys():
        if key[0:3] == 'seq':
            numBuscado = int(key[-2:])
            if row.sorteados_seq.count(numBuscado) == 1:
                if key[3:6] == "Pos":
                    df_seq.loc[idx, key] = df_seq.loc[idx-1, key] + 1 if idx != 0 else 1
                elif key[3:6] == "Neg":
                    df_seq.loc[idx, key] = 0
            else:
                if key[3:6] == "Pos":
                    df_seq.loc[idx, key] = 0
                elif key[3:6] == "Neg":
                    df_seq.loc[idx, key] = df_seq.loc[idx-1, key] + 1 if idx != 0 else 1

df_seq.head()

,Concurso,Data Sorteio,Valor Rateio 15 Números,Valor Rateio 14 Números,Valor Rateio 13 Números,Valor Rateio 12 Números,Valor Rateio 11 Números,sorteados_seq,seqPos01,seqNeg01,...,seqPos21,seqNeg21,seqPos22,seqNeg22,seqPos23,seqNeg23,seqPos24,seqNeg24,seqPos25,seqNeg25
0,1,2003-09-29,49765.82,689.84,10.0,4,2.0,"[2, 3, 5, 6, 9, 10, 11, 13, 14, 16, 18, 20, 23...",0,1,...,0,1,0,1,1,0,1,0,1,0
1,2,2003-10-06,596323.70,1388.95,10.0,4,2.0,"[1, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 19, 20,...",1,0,...,0,2,0,2,2,0,2,0,0,1
2,3,2003-10-13,400623.70,2173.36,10.0,4,2.0,"[1, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 20,...",2,0,...,0,3,0,3,3,0,3,0,0,2
3,4,2003-10-20,902226.02,1498.72,10.0,4,2.0,"[1, 2, 4, 5, 8, 10, 12, 13, 16, 17, 18, 19, 23...",3,0,...,0,4,0,4,4,0,4,0,1,0
4,5,2003-10-27,380017.55,687.49,10.0,4,2.0,"[1, 2, 4, 8, 9, 11, 12, 13, 15, 16, 19, 20, 23...",4,0,...,0,5,0,5,5,0,5,0,2,0


In [6]:
df_clean = df_seq.copy()
for idx, row in df_clean.iterrows():
    for key in row.keys():
        if key[0:3] == 'seq': 
            if idx != df_clean.shape[0]-1:
                if (df_clean.loc[idx, key] < df_clean.loc[idx+1, key]):
                    df_clean.loc[idx, key] = 0

df_clean.head()
            

,Concurso,Data Sorteio,Valor Rateio 15 Números,Valor Rateio 14 Números,Valor Rateio 13 Números,Valor Rateio 12 Números,Valor Rateio 11 Números,sorteados_seq,seqPos01,seqNeg01,...,seqPos21,seqNeg21,seqPos22,seqNeg22,seqPos23,seqNeg23,seqPos24,seqNeg24,seqPos25,seqNeg25
0,1,2003-09-29,49765.82,689.84,10.0,4,2.0,"[2, 3, 5, 6, 9, 10, 11, 13, 14, 16, 18, 20, 23...",0,1,...,0,0,0,0,0,0,0,0,1,0
1,2,2003-10-06,596323.70,1388.95,10.0,4,2.0,"[1, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 19, 20,...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2003-10-13,400623.70,2173.36,10.0,4,2.0,"[1, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 20,...",0,0,...,0,0,0,0,0,0,0,0,0,2
3,4,2003-10-20,902226.02,1498.72,10.0,4,2.0,"[1, 2, 4, 5, 8, 10, 12, 13, 16, 17, 18, 19, 23...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2003-10-27,380017.55,687.49,10.0,4,2.0,"[1, 2, 4, 8, 9, 11, 12, 13, 15, 16, 19, 20, 23...",0,0,...,0,5,0,0,0,0,5,0,0,0


In [55]:

df_stat

,min_Pos,mean_Pos,std_Pos,max_Pos,median_Pos,q05_Pos,q95_Pos,min_Neg,mean_Neg,std_Neg,max_Neg,median_Neg,q05_Neg,q95_Neg
1,1.0,2.407837,1.954637,17.0,2.0,1.0,10.00,1.0,1.624638,0.971911,7.0,1.0,1.0,5.00
2,1.0,2.549923,1.929291,13.0,2.0,1.0,9.50,1.0,1.720430,1.112088,8.0,1.0,1.0,6.00
3,1.0,2.472834,2.029788,18.0,2.0,1.0,10.20,1.0,1.611765,0.998164,7.0,1.0,1.0,5.21
4,1.0,2.442982,1.873475,13.0,2.0,1.0,9.00,1.0,1.621345,0.980011,7.0,1.0,1.0,5.00
5,1.0,2.564688,1.954389,14.0,2.0,1.0,9.00,1.0,1.666667,0.999239,9.0,1.0,1.0,4.44
6,1.0,2.345930,1.850388,14.0,2.0,1.0,9.00,1.0,1.694767,1.046798,7.0,1.0,1.0,5.13
7,1.0,2.279102,1.687949,14.0,2.0,1.0,8.00,1.0,1.619916,1.002100,10.0,1.0,1.0,5.00
8,1.0,2.345985,1.742974,13.0,2.0,1.0,8.16,1.0,1.709913,1.086139,10.0,1.0,1.0,5.00
9,1.0,2.471025,1.844507,13.0,2.0,1.0,9.00,1.0,1.662202,0.995765,8.0,1.0,1.0,5.00
10,1.0,2.596096,2.055663,18.0,2.0,1.0,9.35,1.0,1.580451,0.941879,8.0,1.0,1.0,5.00


In [56]:
concursos = list(df['Concurso'].unique())
proximoJogo = [0]
ganho = 0
gasto = 0
ac11 = 0
ac12 = 0
ac13 = 0
ac14 = 0
ac15 = 0
for concurso in concursos:
    # concurso = df.loc[df.index[-1], "Concurso"]
    idx = df.index[df['Concurso'] == concurso].tolist()[0]
    gasto += 1

    print(f'***** Concurso {df.loc[idx, "Concurso"]:4d}, sorteado em {time.strftime("%d/%m/%Y",  time.strptime(str(df.loc[idx, "Data Sorteio"]), "%Y-%m-%d %H:%M:%S"))} *****', end = '')
    
    # print("Numeros jogados: ", end="")
    # print(*proximoJogo, sep=', ')
    # print('Numeros sorteados: ', end='')
    # print(*df.loc[idx, "sorteados_seq"], sep=", ")
    acertos = 0
    for bola in df.loc[idx, "sorteados_seq"]:
        if bola in proximoJogo:
            acertos += 1
    if acertos > 10:
        coluna = f"Valor Rateio {str(acertos)} Números"
        print(f'Tivemos {acertos} acertos. Ganhamos R$ {df.loc[idx, coluna]:.2f}!!!', end = '\r')
        ganho += df.loc[idx, coluna]
        if acertos == 11:
            ac11 += 1
        if acertos == 12:
            ac12 += 1
        if acertos == 13:
            ac13 += 1
        if acertos == 14:
            ac14 += 1
        if acertos == 15:
            ac15 += 1
    else:
        print(f'Tivemos {acertos} acertos. Não ganhamos =/          ', end = '\r')
    listaFator = []
    listaNum = []
    for numero in range(1,26):
        # print(f'***** Número {numero:2d} *****')
        if numero in df.loc[idx, "sorteados_seq"]:
            # print(f'Geralmente, ele sai por {(df_stat.loc[numero, "q95_Pos"]):.0f} concursos seguidos. ', end = '')
            acumulador = 0
            for i in range(0, 15):
                desloca = max(-i, -idx)
                if numero in df.loc[idx + desloca, "sorteados_seq"]:
                    acumulador += 1
                else:
                    break
            # print(f'Nesse caso, ele saiu nos últimos {acumulador} sorteios.')
            fator_de_acerto = (df_stat.loc[numero, "q95_Pos"] - acumulador) / df_stat.loc[numero, "q95_Pos"]
        else:
            # print(f'Geralmente, ele não sai por {(df_stat.loc[numero, "q95_Neg"]):.0f} concursos seguidos. ', end = '')
            acumulador = 0
            for i in range(1, int(round(df_stat.loc[5, "q95_Neg"],0)*2)):
                desloca = max(-i, -idx)
                if numero in df.loc[idx + desloca, "sorteados_seq"]:
                    acumulador += 1
                else:
                    break
            # print(f'Nesse caso, ele não saiu saiu nos últimos {acumulador} sorteios.')
            fator_de_acerto = acumulador / df_stat.loc[numero, "q95_Neg"]
        # print(f'\nFator = {fator_de_acerto}\n')
        listaFator.append(fator_de_acerto)
        listaNum.append(int(numero))
    listaFull = [listaNum, listaFator]
    dfListaFull = pd.DataFrame(listaFull).T
    dfListaFull.columns = ['Numero','Fator']
    dfListaFull['Numero'] = dfListaFull['Numero'].astype('int')
    dfListaFull.sort_values(by='Fator', inplace=True, ascending=False)
    dfListaFull.reset_index(inplace=True, drop=True)
    proximoJogo = np.sort(list(dfListaFull['Numero'][0:15]))

    # print('-'*50)



In [57]:
    
print(f'Ganho total: R$ {ganho:.2f}. Gasto total: R$ {gasto:.2f}. Rendimento de {(ganho/gasto)-1:.2%}')
print(f'11 acertos: {ac11}')
print(f'12 acertos: {ac12}')
print(f'13 acertos: {ac13}')
print(f'14 acertos: {ac14}')
print(f'15 acertos: {ac15}')

Ganho total: R$ 1280.50. Gasto total: R$ 2780.00. Rendimento de -53.94%
11 acertos: 240
12 acertos: 45
13 acertos: 3
14 acertos: 0
15 acertos: 0


In [58]:
proximoJogo

array([ 3,  4,  7, 10, 11, 12, 13, 14, 15, 17, 19, 21, 22, 23, 24])

In [59]:
listaSorteio = [3,4,5,7,8,10,14,15,16,17,18,20,23,24,25]

In [60]:
acertos = 0
for i in proximoJogo:
    print(f'{i:2.0f}', end=' -> ')
    if i in listaSorteio:
        print("=)")
        acertos += 1
    else:
        print("")
print(f'Acertos: {acertos}')

 3 -> =)
 4 -> =)
 7 -> =)
10 -> =)
11 -> 
12 -> 
13 -> 
14 -> =)
15 -> =)
17 -> =)
19 -> 
21 -> 
22 -> 
23 -> =)
24 -> =)
Acertos: 9
